In [1]:
# Importar bibliotecas
import os
import sys
import pandas as pd
import mlflow
from mlflow.tracking import MlflowClient

# Configurar caminho do MLflow
mlflow.set_tracking_uri('file://' + os.path.abspath('../../mlruns'))

ACTION_REPEAT = 4  # Para Atari

print("✓ Bibliotecas importadas")
print(f"✓ MLflow URI: {mlflow.get_tracking_uri()}")

✓ Bibliotecas importadas
✓ MLflow URI: file:///home/henriquesouza/IA367/IA367-pydreamer/mlruns


In [2]:
# Listar todos os runs disponíveis
client = MlflowClient()

# Experimento 0 é o padrão (local)
experiment_id = '0'

# Buscar todos os runs do experimento
runs = mlflow.search_runs(experiment_ids=[experiment_id])

print(f"Encontrados {len(runs)} runs no experimento {experiment_id}\n")

# Mostrar informações dos runs
for idx, row in runs.iterrows():
    run_id = row['run_id']
    start_time = row['start_time']
    status = row['status']
    env_id = row.get('params.env_id', 'N/A')
    
    print(f"Run {idx + 1}:")
    print(f"  ID: {run_id}")
    print(f"  Ambiente: {env_id}")
    print(f"  Status: {status}")
    print(f"  Início: {start_time}")
    print()

Encontrados 1 runs no experimento 0

Run 1:
  ID: 58d45ccf5a334b369cc2c57fc834332a
  Ambiente: Atari-Pong
  Status: FINISHED
  Início: 2025-11-05 02:38:59.319000+00:00



In [3]:
# Selecionar o run mais recente (normalmente é o que você está treinando)
# Ou você pode escolher um run específico pelo índice

# Para usar o run mais recente:
selected_run = runs.iloc[0]

# OU para escolher manualmente, altere o índice (0, 1, 2, etc.):
# selected_run = runs.iloc[0]  # Primeiro run
# selected_run = runs.iloc[1]  # Segundo run

run_id = selected_run['run_id']
env_id = selected_run.get('params.env_id', 'Atari-Pong')

print(f"✓ Run selecionado: {run_id}")
print(f"✓ Ambiente: {env_id}")

✓ Run selecionado: 58d45ccf5a334b369cc2c57fc834332a
✓ Ambiente: Atari-Pong


In [4]:
# Função para extrair métricas do MLflow
def mlflow_get_metrics(run_id, metric, value_column='y', x_bucket=1):
    """Extrai histórico de uma métrica específica"""
    hist = client.get_metric_history(run_id, metric)
    
    if not hist:
        print(f"Métrica '{metric}' não encontrada")
        return pd.DataFrame()
    
    df = pd.DataFrame({
        'x': [(m.step // x_bucket + 1) * x_bucket for m in hist],
        value_column: [m.value for m in hist],
    })
    return df

# Função para carregar e processar métricas
def load_run_metrics(run_id, env_name, run_name, x_bucket=2000):
    """Carrega métricas principais de um run"""
    
    metrics_to_load = {
        'return': 'agent/return',
        'agent_steps': 'train/data_steps',
    }
    
    df = None
    for k, v in metrics_to_load.items():
        df1 = mlflow_get_metrics(run_id, v, k, x_bucket).groupby('x').mean()
        df = df1 if df is None else df.combine_first(df1)
    
    if df is None or df.empty:
        print(f"Nenhuma métrica encontrada para o run")
        return None
    
    df = df.reset_index().rename(columns={'x': 'train_steps'})
    
    # Calcular env_steps
    if 'agent_steps' in df.columns:
        df['env_steps'] = df['agent_steps'] * ACTION_REPEAT
    
    # Adicionar metadados
    df['env'] = env_name.lower().replace('-', '_')
    df['run'] = run_name
    
    # Remover linhas sem env_steps
    df = df[~df['env_steps'].isna()]
    
    return df

print("Funções definidas")

Funções definidas


In [5]:
# Exportar métricas para CSV
run_name = f"{env_id.lower().replace('-', '_')}_myrun_0"

df_metrics = load_run_metrics(run_id, env_id, run_name, x_bucket=2000)

if df_metrics is not None:
    # Criar diretório runs se não existir
    os.makedirs('runs', exist_ok=True)
    
    # Salvar CSV
    output_file = f'runs/{run_name}.csv'
    df_metrics.round(1).to_csv(output_file, index=False)
    
    print(f"Métricas exportadas para: {output_file}")
    print(f"Total de {len(df_metrics)} registros")
    print(f"\nPreview dos dados:")
    print(df_metrics.head(10))
else:
    print("Não foi possível exportar as métricas")

Métricas exportadas para: runs/atari_pong_myrun_0.csv
Total de 130 registros

Preview dos dados:
   train_steps  agent_steps     return  env_steps         env  \
0         2000      33219.0 -20.830000   132876.0  atari_pong   
1         4000      84577.5 -20.700000   338310.0  atari_pong   
2         6000     120423.5 -20.175000   481694.0  atari_pong   
3         8000     161504.5 -16.733333   646018.0  atari_pong   
4        10000     198894.0 -19.766667   795576.0  atari_pong   
5        12000     236253.0 -13.950000   945012.0  atari_pong   
6        14000     273463.5 -18.100000  1093854.0  atari_pong   
7        16000     307634.5  -7.550000  1230538.0  atari_pong   
8        18000     346562.0  -7.200000  1386248.0  atari_pong   
9        20000     388870.5  -4.800000  1555482.0  atari_pong   

                  run  
0  atari_pong_myrun_0  
1  atari_pong_myrun_0  
2  atari_pong_myrun_0  
3  atari_pong_myrun_0  
4  atari_pong_myrun_0  
5  atari_pong_myrun_0  
6  atari_pong_myrun

Agora que você exportou as métricas para CSV, você pode:

1. **Abrir o notebook `figures.ipynb`** e executar as células para gerar gráficos
2. **Comparar com baselines** do DreamerV2 (os gráficos vão sobrepor automaticamente)
3. **Visualizar a evolução** do retorno médio ao longo do treinamento

O arquivo CSV foi salvo em `runs/` e será automaticamente detectado pelo `figures.ipynb`!